
### 직방의 원룸 매물정보 수집

절차
    - 동이름으로 위도, 경도 수집
    - 위도,경도값 geohash 변환
    - geohash 에서 매물 아이디 수집
    - 매물 아이디에서 매물 정보 수집

In [8]:
!pip install geohash2

  Created wheel for geohash2: filename=geohash2-1.1-py3-none-any.whl size=15544 sha256=1168a6d96da916258604bfff350eb8867a4e804f0ad5d7ec57b6df03f213952d
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\2d\0c\66\dcd768c9e7c26fc81ec59bfe24af9d780fc9dbdf7f90b0cf00
Successfully built geohash2


In [37]:
import requests
import pandas as pd
import geohash2

In [3]:
#1. 동이름으로 위도, 경도 수집

In [18]:
addr = "강남구 역삼동"
URL = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"
response = requests.get(URL)
data = response.json()["items"][0]
lat, lng = data["lat"], data["lng"]
lat, lng

(37.495365142822266, 127.03306579589844)

In [4]:
#2. 위도,경도값 geohash 변환

In [19]:
geohash = geohash2.encode(lat, lng, precision = 5)
geohash

'wydm6'

In [5]:
#3. geohash 에서 매물 아이디 수집

In [28]:
url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&\
geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
response = requests.get(url)
data = response.json()["items"]
ids = [data["item_id"] for data in data]
ids[:5]

[32819635, 32838798, 32872608, 32838700, 32767020]

In [6]:
#4. 매물 아이디에서 매물 정보 수집

In [45]:
url = "https://apis.zigbang.com/v2/items/list"
# post 방식이라 params 추가 (get방식이면 상관없음)
params = {
    "domain": "zigbang",
    "withCoalition": "true",
    "item_ids": ids[:900]
}
response = requests.post(url, params)
items = response.json()["items"]
items[:2]
df = pd.DataFrame(items)
df[["item_id","sales_type","deposit",'rent','size_m2','address1','manage_cost']].head(10)

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32819635,전세,16500,0,39.89,서울시 서초구 서초동,7.5
1,32838798,월세,1000,68,37.56,서울시 서초구 서초동,7.5
2,32872608,월세,1000,70,37.56,서울시 서초구 서초동,7.5
3,32838700,전세,50400,0,66.12,서울시 서초구 서초동,9
4,32767020,월세,20000,90,60.12,서울시 서초구 서초동,5
5,32830406,월세,1000,65,59.50,서울시 서초구 서초동,15
6,32862728,월세,1000,50,49.59,서울시 서초구 서초동,14
7,32866406,전세,52000,0,37.63,서울시 서초구 서초동,10
8,32432677,전세,12000,0,47.93,서울시 서초구 서초동,14
9,32578425,전세,12900,0,39.67,서울시 서초구 서초동,12


In [58]:
def oneroom(addr):
    URL = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"
    response = requests.get(URL)
    data = response.json()["items"][0]
    lat, lng = data["lat"], data["lng"]
    
    geohash = geohash2.encode(lat, lng, precision = 5)
    
    url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&\
geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
    response = requests.get(url)
    data2 = response.json()["items"]
    ids = [data2["item_id"] for data2 in data2]
    
    url = "https://apis.zigbang.com/v2/items/list"
    params = {
        "domain": "zigbang",
        "withCoalition": "true",
        "item_ids": ids[:900]
    }
    response = requests.post(url, params)
    items = response.json()["items"]
    df = pd.DataFrame(items)
    df = df[["item_id","sales_type","deposit",'rent','size_m2','address1','manage_cost']].head(10)
    return df

In [59]:
addr = "마포구 합정동"
df = oneroom(addr)
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
8,32855914,월세,2000,40,19.83,서울시 영등포구 당산동,7
9,32788937,월세,500,45,23.14,서울시 영등포구 당산동4가,0
